# 3/20
- やったこと
    - 	age	index_group_no	top12のテーブルを作った

In [1]:
import pandas as pd
import numpy as np

In [2]:
# データの読み込み
DATA_DIR = "../data/original/"
articles_df=pd.read_pickle(f'{DATA_DIR}articles.pkl')
customers_df=pd.read_pickle(f'{DATA_DIR}customers.pkl')
transactions_train_df=pd.read_pickle(f'{DATA_DIR}transactions_train.pkl')

In [3]:
#articles_df = articles_df.head(10000)
#customers_df = customers_df.head(10000)
#transactions_train_df = transactions_train_df.head(10000)

In [4]:
# articles と customersをtransactionsを使って結合する
def create_features(articles_df,customers_df,transactions_train_df):
    tt_temp = transactions_train_df[["customer_id","article_id"]]
    a_df = articles_df[["article_id","index_group_no"]]
    c_df = customers_df[["customer_id","age"]]
    at_merged_df =  pd.merge(tt_temp,a_df)
    features_df = pd.merge(c_df,at_merged_df)
    features_df = features_df.dropna(how="all")
    return features_df

In [5]:
# １０までとってくる
def convert_age_into_10_delimited(df):
    df["age"] = df["age"] / 10
    df["age"] = df["age"].astype(np.int64) 
    df["age"] = df["age"] * 10
    return df

In [6]:
features_df = create_features(articles_df,customers_df,transactions_train_df)

In [7]:
# ageの欠損値を0で埋める
def fillna_to_zero_in_age(df):
    df["age"] = df["age"].fillna(0)
    return df
features_df = fillna_to_zero_in_age(features_df)

In [8]:
features_df = convert_age_into_10_delimited(features_df)

In [9]:
# top 12のarticles_id
articles_most_buy_30 = list(features_df['article_id'].value_counts().head(30).index)

In [14]:
df = features_df

# ageとindex?group

In [15]:
count = 0

df = df.groupby(["age","index_group_no"])["article_id"].agg(lambda x:list(x)).reset_index()
df

,age,index_group_no,article_id
0,0,1,"[656719005, 568601006, 568601006, 607642008, 7..."
1,0,2,"[745232001, 890498002, 377277001, 640244003, 6..."
2,0,3,"[176209023, 727808001, 727808007, 852643003, 8..."
3,0,4,"[627759010, 625548001, 697138006, 694736004, 7..."
4,0,26,"[863103001, 794321007, 732413001, 742079001, 7..."


In [16]:
# 上位12のarticle idをとってきておく
TOP_N = 12
def calc_top(ids):
    ids= sorted(ids)
    if len(ids) < TOP_N:
        return [*ids, *articles_most_buy_30][0:TOP_N]
    else:
        return ids[0:TOP_N]
df[[f"top{i}" for i in range(1,TOP_N+1)]] = df.apply(lambda x: calc_top(x["article_id"]), axis = 1,result_type='expand')
df = df.drop(["article_id"],axis=1)

In [17]:
df.to_pickle()

,age,index_group_no,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12
0,0,1,108775015,108775015,108775015,108775015,108775015,108775015,108775015,108775015,108775015,108775015,108775015,108775015
1,0,2,162074062,162074062,162074062,162074062,162074062,162074062,162074062,162074062,162074062,162074062,162074062,162074062
2,0,3,118458003,118458003,118458003,118458003,118458003,118458003,118458003,118458003,118458003,118458003,118458003,118458003
3,0,4,112679048,112679048,112679048,112679052,112679052,112679052,141661022,141661022,141661025,146706001,146706001,146706001
4,0,26,145872001,145872001,145872001,145872001,145872001,145872001,145872001,145872001,145872001,145872001,145872001,145872001
